# 🌤️ Weather Model Training - Dual Model (Hourly + Daily)

Notebook ini mengikuti panduan dari `training_guide.md` untuk melatih **dual-model**:
1. **Model Hourly** - Prediksi per-jam (temp, humidity, windspeed, pressure, weather_code)
2. **Model Daily** - Prediksi per-hari (temp_min, temp_max, temp_mean, humidity_avg, windspeed_avg, pressure_avg, weather_code_dominant)

**Output:** 7 file model `.pkl` untuk berbagai kebutuhan deployment.

## 1. Persiapan Lingkungan dan Pemuatan Pustaka

In [ ]:
# Install dependencies jika belum ada
# !pip install pandas numpy matplotlib seaborn scikit-learn xgboost joblib

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, f1_score, classification_report, confusion_matrix
)

# Regression Models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Classification Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# XGBoost
try:
    from xgboost import XGBRegressor, XGBClassifier
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not installed. Skipping XGBoost models.")
    XGBOOST_AVAILABLE = False

# Joblib for saving models
import joblib

print("✅ Semua pustaka berhasil diimpor!")
print(f"   - Pandas: {pd.__version__}")
print(f"   - NumPy: {np.__version__}")
print(f"   - XGBoost Available: {XGBOOST_AVAILABLE}")

## 2. Pengumpulan dan Pemuatan Data

In [ ]:
# Load dataset (23 kolom: hourly + daily features)
DATA_PATH = '../data/historical_data_2000_2024.csv'

df = pd.read_csv(DATA_PATH)

# Konversi timestamp ke datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Urutkan berdasarkan waktu (PENTING untuk time-series)
df = df.sort_values('timestamp').reset_index(drop=True)

print(f"📊 Dataset loaded: {len(df):,} baris x {len(df.columns)} kolom")
print(f"📅 Rentang waktu: {df['timestamp'].min()} - {df['timestamp'].max()}")
print(f"\n📋 Kolom dataset:")
print(df.columns.tolist())
df.head()

In [ ]:
# Info struktur data
df.info()

## 3. Analisis Data Eksplorasi (EDA)

### 3.1 Statistik Deskriptif

In [ ]:
# Statistik deskriptif untuk fitur numerik
df.describe()

### 3.2 Visualisasi Distribusi

In [ ]:
# Visualisasi distribusi parameter cuaca utama (Hourly)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

params = ['temp', 'humidity', 'windspeed', 'sealevelpressure']
titles = ['Temperature (°C)', 'Humidity (%)', 'Wind Speed (km/h)', 'Sea Level Pressure (hPa)']

for ax, param, title in zip(axes.flatten(), params, titles):
    sns.histplot(df[param], kde=True, ax=ax, color='steelblue')
    ax.set_title(f'Distribusi {title}')
    ax.set_xlabel(title)

plt.suptitle('Distribusi Parameter Cuaca Hourly', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Visualisasi distribusi parameter cuaca Daily
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

daily_params = ['temp_max_daily', 'temp_min_daily', 'temp_mean_daily', 
                'humidity_avg_daily', 'pressure_avg_daily', 'windspeed_avg_daily']
titles = ['Temp Max (°C)', 'Temp Min (°C)', 'Temp Mean (°C)', 
          'Humidity Avg (%)', 'Pressure Avg (hPa)', 'Windspeed Avg (km/h)']

for ax, param, title in zip(axes.flatten(), daily_params, titles):
    sns.histplot(df[param].dropna(), kde=True, ax=ax, color='coral')
    ax.set_title(f'Distribusi {title}')
    ax.set_xlabel(title)

plt.suptitle('Distribusi Parameter Cuaca Daily', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 3.3 Analisis Korelasi

In [ ]:
# Heatmap korelasi
numeric_cols = ['temp', 'humidity', 'windspeed', 'sealevelpressure', 'rain', 
                'weather_code', 'temp_max_daily', 'temp_min_daily', 'temp_mean_daily',
                'humidity_avg_daily', 'pressure_avg_daily', 'windspeed_avg_daily']

plt.figure(figsize=(14, 10))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Heatmap Korelasi Antar Variabel Cuaca (Hourly + Daily)')
plt.tight_layout()
plt.show()

### 3.4 Analisis Korelasi: weather_code dan rain

In [ ]:
# Analisis hubungan weather_code dengan rain
weather_rain_analysis = df.groupby('weather_code')[['rain']].agg(['mean', 'min', 'max', 'count'])
print("📊 Weather Code vs Rain:")
weather_rain_analysis

In [ ]:
# Verifikasi korelasi deterministik
print(f"\n🔍 Verifikasi rain == precipitation: {(df['rain'] == df['precipitation']).all()}")
print(f"🔍 Weather codes dengan rain > 0: {sorted(df[df['rain'] > 0]['weather_code'].unique())}")
print(f"🔍 Weather codes dengan rain = 0: {sorted(df[df['rain'] == 0]['weather_code'].unique())}")

# Kesimpulan
print("\n✅ KESIMPULAN:")
print("   - rain dan precipitation IDENTIK di seluruh dataset")
print("   - weather_code >= 50 SELALU hujan (deterministik)")
print("   - Tidak perlu memprediksi rain terpisah, cukup prediksi weather_code")

## 4. Pra-pemrosesan Data dan Feature Engineering

### 4.1 Preprocessing Data Hourly

In [ ]:
# Copy dataframe untuk preprocessing
df_hourly = df.copy()

# 1. Label Encoding untuk 'conditions'
le_conditions = LabelEncoder()
df_hourly['conditions_encoded'] = le_conditions.fit_transform(df_hourly['conditions'])

print("📝 Label Encoding untuk 'conditions':")
for i, label in enumerate(le_conditions.classes_):
    print(f"   {i}: {label}")

In [ ]:
# 2. Feature Engineering: Lag Features untuk Hourly
hourly_target_cols = ['temp', 'humidity', 'windspeed', 'sealevelpressure']

for col in hourly_target_cols:
    # Lag 1 jam
    df_hourly[f'{col}_lag_1'] = df_hourly[col].shift(1)
    # Lag 24 jam
    df_hourly[f'{col}_lag_24'] = df_hourly[col].shift(24)
    # Rolling mean 24 jam
    df_hourly[f'{col}_rolling_24'] = df_hourly[col].rolling(window=24).mean()

print(f"✅ Feature Engineering Hourly selesai! Kolom baru: {12} fitur lag & rolling")

In [ ]:
# 3. Hapus baris dengan NaN (akibat lag & rolling)
rows_before = len(df_hourly)
df_hourly = df_hourly.dropna().reset_index(drop=True)
rows_after = len(df_hourly)

print(f"🗑️ Baris dihapus (NaN): {rows_before - rows_after:,}")
print(f"📊 Dataset Hourly final: {rows_after:,} baris")

### 4.2 Preprocessing Data Daily

In [ ]:
# Agregasi data hourly menjadi daily
df_daily = df.groupby(['year', 'month', 'day']).agg({
    'temp': ['min', 'max', 'mean'],
    'humidity': 'mean',
    'windspeed': 'mean',
    'sealevelpressure': 'mean',
    'weather_code': lambda x: x.mode()[0],  # Dominan weather_code
    'rain': 'sum'  # Total curah hujan
}).reset_index()

# Flatten column names
df_daily.columns = ['year', 'month', 'day', 
                    'temp_min', 'temp_max', 'temp_mean',
                    'humidity_avg', 'windspeed_avg', 'pressure_avg',
                    'weather_code_dominant', 'rain_total']

print(f"📊 Dataset Daily: {len(df_daily):,} baris (hari)")
df_daily.head()

In [ ]:
# Feature Engineering Daily - Lag Features
df_daily['temp_min_lag_1'] = df_daily['temp_min'].shift(1)   # Kemarin
df_daily['temp_max_lag_1'] = df_daily['temp_max'].shift(1)
df_daily['temp_mean_lag_1'] = df_daily['temp_mean'].shift(1)
df_daily['humidity_avg_lag_1'] = df_daily['humidity_avg'].shift(1)
df_daily['windspeed_avg_lag_1'] = df_daily['windspeed_avg'].shift(1)
df_daily['pressure_avg_lag_1'] = df_daily['pressure_avg'].shift(1)

df_daily['temp_min_lag_7'] = df_daily['temp_min'].shift(7)   # Seminggu lalu
df_daily['temp_max_lag_7'] = df_daily['temp_max'].shift(7)
df_daily['temp_mean_lag_7'] = df_daily['temp_mean'].shift(7)
df_daily['rain_total_lag_1'] = df_daily['rain_total'].shift(1)

# Hapus NaN
rows_before = len(df_daily)
df_daily = df_daily.dropna().reset_index(drop=True)
rows_after = len(df_daily)

print(f"🗑️ Baris dihapus (NaN): {rows_before - rows_after:,}")
print(f"📊 Dataset Daily final: {rows_after:,} baris")

## 5. Pelatihan dan Perbandingan Model

### 5.1 Pemisahan Data (Time-Series Split)

In [ ]:
# ===== HOURLY DATA SPLIT =====
hourly_train_size = int(len(df_hourly) * 0.8)
hourly_train = df_hourly[:hourly_train_size]
hourly_test = df_hourly[hourly_train_size:]

print(f"📊 HOURLY Data Split (80-20):")
print(f"   Train: {len(hourly_train):,} baris")
print(f"   Test:  {len(hourly_test):,} baris")

# ===== DAILY DATA SPLIT =====
daily_train_size = int(len(df_daily) * 0.8)
daily_train = df_daily[:daily_train_size]
daily_test = df_daily[daily_train_size:]

print(f"\n📊 DAILY Data Split (80-20):")
print(f"   Train: {len(daily_train):,} baris")
print(f"   Test:  {len(daily_test):,} baris")

In [ ]:
# ===== DEFINISI FITUR DAN TARGET =====

# HOURLY Features
hourly_feature_cols = [
    'year', 'month', 'day', 'hour',
    'temp_lag_1', 'temp_lag_24', 'temp_rolling_24',
    'humidity_lag_1', 'humidity_lag_24', 'humidity_rolling_24',
    'windspeed_lag_1', 'windspeed_lag_24', 'windspeed_rolling_24',
    'sealevelpressure_lag_1', 'sealevelpressure_lag_24', 'sealevelpressure_rolling_24'
]
hourly_target_reg = ['temp', 'humidity', 'windspeed', 'sealevelpressure']
hourly_target_clf = 'weather_code'

# DAILY Features
daily_feature_cols = [
    'year', 'month', 'day',
    'temp_min_lag_1', 'temp_max_lag_1', 'temp_mean_lag_1',
    'humidity_avg_lag_1', 'windspeed_avg_lag_1', 'pressure_avg_lag_1',
    'temp_min_lag_7', 'temp_max_lag_7', 'temp_mean_lag_7',
    'rain_total_lag_1'
]
daily_target_reg = ['temp_min', 'temp_max', 'temp_mean', 'humidity_avg', 'windspeed_avg', 'pressure_avg']
daily_target_clf = 'weather_code_dominant'

# Pisahkan X dan y untuk HOURLY
X_hourly_train = hourly_train[hourly_feature_cols]
X_hourly_test = hourly_test[hourly_feature_cols]
y_hourly_train_reg = hourly_train[hourly_target_reg]
y_hourly_test_reg = hourly_test[hourly_target_reg]
y_hourly_train_clf = hourly_train[hourly_target_clf]
y_hourly_test_clf = hourly_test[hourly_target_clf]

# Pisahkan X dan y untuk DAILY
X_daily_train = daily_train[daily_feature_cols]
X_daily_test = daily_test[daily_feature_cols]
y_daily_train_reg = daily_train[daily_target_reg]
y_daily_test_reg = daily_test[daily_target_reg]
y_daily_train_clf = daily_train[daily_target_clf]
y_daily_test_clf = daily_test[daily_target_clf]

print(f"✅ HOURLY Features: {len(hourly_feature_cols)} | Targets Reg: {hourly_target_reg} | Target Clf: {hourly_target_clf}")
print(f"✅ DAILY Features: {len(daily_feature_cols)} | Targets Reg: {daily_target_reg} | Target Clf: {daily_target_clf}")

### 5.2 Komparasi Model Regresi (Hourly)

In [ ]:
# Definisi model regresi
regression_models = {
    'Linear Regression': LinearRegression(),
    'K-Neighbors': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=10),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
}

if XGBOOST_AVAILABLE:
    regression_models['XGBoost'] = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)

# Fungsi evaluasi
def evaluate_regression(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2}

In [ ]:
# Training HOURLY Regression Models
print("="*70)
print("🔄 TRAINING HOURLY REGRESSION MODELS")
print("="*70)

hourly_reg_results = []

for name, model in regression_models.items():
    print(f"\n🔄 Training {name}...")
    model.fit(X_hourly_train, y_hourly_train_reg)
    y_pred = model.predict(X_hourly_test)
    
    overall_metrics = evaluate_regression(y_hourly_test_reg, y_pred)
    overall_metrics['Model'] = name
    hourly_reg_results.append(overall_metrics)
    
    print(f"   ✅ {name} - R²: {overall_metrics['R2']:.4f}, RMSE: {overall_metrics['RMSE']:.4f}")

df_hourly_reg = pd.DataFrame(hourly_reg_results).sort_values('R2', ascending=False)
print("\n📊 HASIL PERBANDINGAN REGRESI HOURLY:")
display(df_hourly_reg[['Model', 'R2', 'RMSE', 'MAE']].reset_index(drop=True))

best_hourly_reg_model = df_hourly_reg.iloc[0]['Model']
print(f"\n🏆 MODEL TERBAIK REGRESI HOURLY: {best_hourly_reg_model}")

### 5.3 Komparasi Model Klasifikasi (Hourly)

In [ ]:
# Definisi model klasifikasi
classification_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
}

if XGBOOST_AVAILABLE:
    classification_models['XGBoost'] = XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)

In [ ]:
# Training HOURLY Classification Models
print("="*70)
print("🔄 TRAINING HOURLY CLASSIFICATION MODELS")
print("="*70)

hourly_clf_results = []

for name, model in classification_models.items():
    print(f"\n🔄 Training {name}...")
    model.fit(X_hourly_train, y_hourly_train_clf)
    y_pred = model.predict(X_hourly_test)
    
    accuracy = accuracy_score(y_hourly_test_clf, y_pred)
    f1_weighted = f1_score(y_hourly_test_clf, y_pred, average='weighted', zero_division=0)
    
    hourly_clf_results.append({
        'Model': name,
        'Accuracy': accuracy,
        'F1 (Weighted)': f1_weighted
    })
    
    print(f"   ✅ {name} - Accuracy: {accuracy:.4f}, F1: {f1_weighted:.4f}")

df_hourly_clf = pd.DataFrame(hourly_clf_results).sort_values('Accuracy', ascending=False)
print("\n📊 HASIL PERBANDINGAN KLASIFIKASI HOURLY:")
display(df_hourly_clf.reset_index(drop=True))

best_hourly_clf_model = df_hourly_clf.iloc[0]['Model']
print(f"\n🏆 MODEL TERBAIK KLASIFIKASI HOURLY: {best_hourly_clf_model}")

### 5.4 Komparasi Model Regresi (Daily)

In [ ]:
# Training DAILY Regression Models
print("="*70)
print("🔄 TRAINING DAILY REGRESSION MODELS")
print("="*70)

daily_reg_results = []

# Re-initialize models
regression_models_daily = {
    'Linear Regression': LinearRegression(),
    'K-Neighbors': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=10),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
}
if XGBOOST_AVAILABLE:
    regression_models_daily['XGBoost'] = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)

for name, model in regression_models_daily.items():
    print(f"\n🔄 Training {name}...")
    model.fit(X_daily_train, y_daily_train_reg)
    y_pred = model.predict(X_daily_test)
    
    overall_metrics = evaluate_regression(y_daily_test_reg, y_pred)
    overall_metrics['Model'] = name
    daily_reg_results.append(overall_metrics)
    
    print(f"   ✅ {name} - R²: {overall_metrics['R2']:.4f}, RMSE: {overall_metrics['RMSE']:.4f}")

df_daily_reg = pd.DataFrame(daily_reg_results).sort_values('R2', ascending=False)
print("\n📊 HASIL PERBANDINGAN REGRESI DAILY:")
display(df_daily_reg[['Model', 'R2', 'RMSE', 'MAE']].reset_index(drop=True))

best_daily_reg_model = df_daily_reg.iloc[0]['Model']
print(f"\n🏆 MODEL TERBAIK REGRESI DAILY: {best_daily_reg_model}")

### 5.5 Komparasi Model Klasifikasi (Daily)

In [ ]:
# Training DAILY Classification Models
print("="*70)
print("🔄 TRAINING DAILY CLASSIFICATION MODELS")
print("="*70)

daily_clf_results = []

# Re-initialize models
classification_models_daily = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
}
if XGBOOST_AVAILABLE:
    classification_models_daily['XGBoost'] = XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)

for name, model in classification_models_daily.items():
    print(f"\n🔄 Training {name}...")
    model.fit(X_daily_train, y_daily_train_clf)
    y_pred = model.predict(X_daily_test)
    
    accuracy = accuracy_score(y_daily_test_clf, y_pred)
    f1_weighted = f1_score(y_daily_test_clf, y_pred, average='weighted', zero_division=0)
    
    daily_clf_results.append({
        'Model': name,
        'Accuracy': accuracy,
        'F1 (Weighted)': f1_weighted
    })
    
    print(f"   ✅ {name} - Accuracy: {accuracy:.4f}, F1: {f1_weighted:.4f}")

df_daily_clf = pd.DataFrame(daily_clf_results).sort_values('Accuracy', ascending=False)
print("\n📊 HASIL PERBANDINGAN KLASIFIKASI DAILY:")
display(df_daily_clf.reset_index(drop=True))

best_daily_clf_model = df_daily_clf.iloc[0]['Model']
print(f"\n🏆 MODEL TERBAIK KLASIFIKASI DAILY: {best_daily_clf_model}")

### 5.6 Kesimpulan Pemilihan Model

In [ ]:
print("="*70)
print("🎯 KESIMPULAN PEMILIHAN MODEL TERBAIK")
print("="*70)

print("\n📈 MODEL HOURLY:")
print(f"   🏆 Regresi: {best_hourly_reg_model} (R²: {df_hourly_reg.iloc[0]['R2']:.4f})")
print(f"   🏆 Klasifikasi: {best_hourly_clf_model} (Acc: {df_hourly_clf.iloc[0]['Accuracy']:.4f})")

print("\n📊 MODEL DAILY:")
print(f"   🏆 Regresi: {best_daily_reg_model} (R²: {df_daily_reg.iloc[0]['R2']:.4f})")
print(f"   🏆 Klasifikasi: {best_daily_clf_model} (Acc: {df_daily_clf.iloc[0]['Accuracy']:.4f})")

print("\n" + "="*70)
print("✅ Langkah selanjutnya: Retraining dengan 100% data, lalu simpan 7 file model")
print("="*70)

## 6. Retraining dengan 100% Data dan Penyimpanan Model

In [ ]:
# Gabung train + test untuk final training
X_hourly_full = df_hourly[hourly_feature_cols]
y_hourly_reg_full = df_hourly[hourly_target_reg]
y_hourly_clf_full = df_hourly[hourly_target_clf]

X_daily_full = df_daily[daily_feature_cols]
y_daily_reg_full = df_daily[daily_target_reg]
y_daily_clf_full = df_daily[daily_target_clf]

print(f"📊 Full Hourly Data: {len(X_hourly_full):,} baris")
print(f"📊 Full Daily Data: {len(X_daily_full):,} baris")

In [ ]:
# Final Training - Hourly Models (menggunakan Random Forest sebagai default)
print("🔄 Training Final Models...\n")

# HOURLY
hourly_regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
hourly_regressor.fit(X_hourly_full, y_hourly_reg_full)
print("✅ Hourly Regressor trained")

hourly_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
hourly_classifier.fit(X_hourly_full, y_hourly_clf_full)
print("✅ Hourly Classifier trained")

# DAILY
daily_regressor = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
daily_regressor.fit(X_daily_full, y_daily_reg_full)
print("✅ Daily Regressor trained")

daily_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
daily_classifier.fit(X_daily_full, y_daily_clf_full)
print("✅ Daily Classifier trained")

In [ ]:
# Metadata
weather_code_to_rain = {0:0, 1:0, 2:0, 3:0, 51:0.2, 53:0.7, 55:1.1, 61:1.7, 63:4.0, 65:10.3}

# ===== PACKAGE DEFINITIONS =====

# 1. Combined Package
combined_package = {
    'hourly': {
        'regressor': hourly_regressor,
        'classifier': hourly_classifier,
        'feature_columns': hourly_feature_cols,
        'target_regression': hourly_target_reg,
        'target_classification': hourly_target_clf,
    },
    'daily': {
        'regressor': daily_regressor,
        'classifier': daily_classifier,
        'feature_columns': daily_feature_cols,
        'target_regression': daily_target_reg,
        'target_classification': daily_target_clf,
    },
    'label_encoder': le_conditions,
    'weather_code_to_rain': weather_code_to_rain,
    'version': '2.1',
    'trained_date': datetime.now().isoformat(),
    'model_type': 'combined'
}

# 2. Hourly Package
hourly_package = {
    'regressor': hourly_regressor,
    'classifier': hourly_classifier,
    'feature_columns': hourly_feature_cols,
    'target_regression': hourly_target_reg,
    'target_classification': hourly_target_clf,
    'label_encoder': le_conditions,
    'weather_code_to_rain': weather_code_to_rain,
    'version': '2.1',
    'trained_date': datetime.now().isoformat(),
    'model_type': 'hourly'
}

# 3. Daily Package
daily_package = {
    'regressor': daily_regressor,
    'classifier': daily_classifier,
    'feature_columns': daily_feature_cols,
    'target_regression': daily_target_reg,
    'target_classification': daily_target_clf,
    'label_encoder': le_conditions,
    'weather_code_to_rain': weather_code_to_rain,
    'version': '2.1',
    'trained_date': datetime.now().isoformat(),
    'model_type': 'daily'
}

# 4-7. Individual Packages
hourly_reg_package = {
    'model': hourly_regressor,
    'feature_columns': hourly_feature_cols,
    'target': hourly_target_reg,
    'version': '2.1',
    'model_type': 'hourly_regressor'
}

hourly_clf_package = {
    'model': hourly_classifier,
    'feature_columns': hourly_feature_cols,
    'target': hourly_target_clf,
    'label_encoder': le_conditions,
    'weather_code_to_rain': weather_code_to_rain,
    'version': '2.1',
    'model_type': 'hourly_classifier'
}

daily_reg_package = {
    'model': daily_regressor,
    'feature_columns': daily_feature_cols,
    'target': daily_target_reg,
    'version': '2.1',
    'model_type': 'daily_regressor'
}

daily_clf_package = {
    'model': daily_classifier,
    'feature_columns': daily_feature_cols,
    'target': daily_target_clf,
    'label_encoder': le_conditions,
    'weather_code_to_rain': weather_code_to_rain,
    'version': '2.1',
    'model_type': 'daily_classifier'
}

print("✅ All model packages created!")

In [ ]:
# Simpan 7 file model
MODEL_DIR = 'models'
os.makedirs(MODEL_DIR, exist_ok=True)

# 1-3: Main packages
joblib.dump(combined_package, f'{MODEL_DIR}/weather_model_combined.pkl')
print("✅ weather_model_combined.pkl saved")

joblib.dump(hourly_package, f'{MODEL_DIR}/weather_model_hourly.pkl')
print("✅ weather_model_hourly.pkl saved")

joblib.dump(daily_package, f'{MODEL_DIR}/weather_model_daily.pkl')
print("✅ weather_model_daily.pkl saved")

# 4-7: Individual packages
joblib.dump(hourly_reg_package, f'{MODEL_DIR}/weather_model_hourly_regressor.pkl')
print("✅ weather_model_hourly_regressor.pkl saved")

joblib.dump(hourly_clf_package, f'{MODEL_DIR}/weather_model_hourly_classifier.pkl')
print("✅ weather_model_hourly_classifier.pkl saved")

joblib.dump(daily_reg_package, f'{MODEL_DIR}/weather_model_daily_regressor.pkl')
print("✅ weather_model_daily_regressor.pkl saved")

joblib.dump(daily_clf_package, f'{MODEL_DIR}/weather_model_daily_classifier.pkl')
print("✅ weather_model_daily_classifier.pkl saved")

print(f"\n🎉 Total: 7 model files saved to {MODEL_DIR}/")

## Rangkuman

Notebook ini telah menyelesaikan:

1. ✅ **Persiapan Lingkungan** - Import semua pustaka
2. ✅ **Pemuatan Data** - Load dataset 23 kolom (hourly + daily features)
3. ✅ **EDA** - Analisis distribusi, korelasi, hubungan weather_code dengan rain
4. ✅ **Feature Engineering** - Lag features untuk Hourly dan Daily
5. ✅ **Perbandingan Model** - 5 model regresi & 4 model klasifikasi untuk Hourly dan Daily
6. ✅ **Retraining & Penyimpanan** - 7 file model `.pkl` tersimpan

**Output Files:**
```
models/
├── weather_model_combined.pkl           # Hourly + Daily
├── weather_model_hourly.pkl             # Hourly only
├── weather_model_daily.pkl              # Daily only
├── weather_model_hourly_regressor.pkl   # Hourly Reg only
├── weather_model_hourly_classifier.pkl  # Hourly Clf only
├── weather_model_daily_regressor.pkl    # Daily Reg only
└── weather_model_daily_classifier.pkl   # Daily Clf only
```